In [11]:
import numpy as np
import scipy
from scipy import io
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline

In [12]:
data = scipy.io.loadmat('hw5_data/spam_data/spam_data.mat')

In [13]:
X_train = data['training_data']
X_test = data['test_data']
labels_train = data['training_labels']
labels_train = labels_train.reshape((labels_train.shape[1],))

In [14]:
X_train.shape

(5172, 242)

In [15]:
"""Randomly selecting 4172 training points and 1000 validation points"""
train_idx = np.random.choice(X_train.shape[0], X_train.shape[0]-1000, replace=False)
val_idx = np.delete(np.arange(X_train.shape[0]), train_idx)

X_train_cut = X_train[train_idx]
labels_train_cut = labels_train[train_idx]

X_val = X_train[val_idx]
labels_val = labels_train[val_idx]

In [119]:
dt = DecisionTree(15, 0.1)
dt.train(X_train_cut, labels_train_cut)

In [120]:
pred_labels_train = np.array([dt.predict(X_train_cut[i]) for i in range(X_train_cut.shape[0])])
pred_labels_val = np.array([dt.predict(X_val[i]) for i in range(X_val.shape[0])])

In [121]:
import sklearn.metrics as metrics
print("Train accuracy: {0}".format(metrics.accuracy_score(labels_train_cut, pred_labels_train)))
print("Val accuracy: {0}".format(metrics.accuracy_score(labels_val, pred_labels_val)))

Train accuracy: 0.9225790987535955
Val accuracy: 0.895


In [570]:
"""For Final Submission"""
dt_final = DecisionTree(15, 0.1)
dt_final.train(X_train, labels_train)
pred_labels_train = np.array([dt.predict(X_train[i]) for i in range(X_train.shape[0])])
pred_labels_test = np.array([dt.predict(X_test[i]) for i in range(X_test.shape[0])])
print("Train accuracy: {0}".format(metrics.accuracy_score(labels_train, pred_labels_train)))

Train accuracy: 0.9098994586233565


In [521]:
"""CONTEST SUBMISSION"""
import csv
f = open("ashton_teng_spam_predictions.csv", 'wt')
try:
    writer = csv.writer(f)
    writer.writerow(('Id', 'Category'))
    for i in range(len(pred_labels_test)):
        writer.writerow( (i+1, pred_labels_test[i]) )
finally:
    f.close()

In [19]:
class Node:
    def __init__(self, depth):
        self.depth = depth
        self.is_leaf = False
        self.split_rule = None
    def set_left(self, left_node):
        self.left = left_node
    def get_left(self):
        return self.left
    def set_right(self, right_node):
        self.right = right_node
    def get_right(self):
        return self.right
    def set_split_rule(self, split_rule):
        self.split_rule = split_rule
    def get_split_rule(self):
        return self.split_rule
    def set_counts(self, count1, count0):
        self.count1 = count1
        self.count0 = count0
    def get_counts(self):
        return self.count1, self.count0

In [20]:
class DecisionTree:
    
    def __init__(self, max_depth, entropy_threshold):
        self.root_node = None
        self.max_depth = max_depth
        self.entropy_threshold = entropy_threshold
    
    def impurity(self, left_1_count, left_0_count, right_1_count, right_0_count):
        """count the frequencies of labels on the ”left” and ”right” side of that split. 
         The method calculates and outputs a scalar value representing the impurity (i.e. the ”badness”)
         of the specified split on the input data."""
        H_left = self.entropy(left_1_count, left_0_count)
        H_right = self.entropy(right_1_count, right_0_count)
        left_count = left_1_count + left_0_count
        right_count = right_1_count + right_0_count
        return (left_count*H_left+right_count*H_right) / (left_count+right_count)
        
    def segmenter(self, data, labels):
        """finds the best split rule for a Node"""
        #try all split_rules and send histograms to impurity
        import sys
        best_impurity = sys.maxsize
        best_split_rule = None
        for feature in range(data.shape[1]):
            feature_data = data[:,feature]
            best_local_impurity = sys.maxsize
            best_local_threshold = None
            feature_set = set(feature_data)
            if len(feature_set) <= 1:
                continue
            if len(feature_set) == 2:
                feature_set = {sum(feature_set)/2}
            else:
                feature_set.remove(min(feature_set))
            for threshold in feature_set:
                left_data, left_labels, right_data, right_labels = self.splitData((feature, threshold), data, labels)
                left_1_count = sum(left_labels)
                left_0_count = len(left_labels) - left_1_count
                right_1_count = sum(right_labels)
                right_0_count = len(right_labels) - right_1_count
                impurity = self.impurity(left_1_count, left_0_count, right_1_count, right_0_count)
                if impurity == 0:
                    return (feature, threshold)
                if impurity < best_local_impurity:
                    best_local_impurity = impurity
                    best_local_threshold = threshold
            if best_local_impurity < best_impurity:
                best_impurity = best_local_impurity
                best_split_rule = (feature, best_local_threshold) 
        return best_split_rule
    
    def entropy(self, count1, count0):
        total = count0+count1
        p0 = count0/total
        p1 = count1/total
        entropy = 0
        if p0 > 0:
            entropy -= p0*np.log(p0)
        if p1 > 0:
            entropy -= p1*np.log(p1)
        return entropy
    
    def splitData(self, split_rule, data, labels):
        feature, threshold = split_rule
        feature_data = data[:,feature]
        left_idx = np.argwhere(feature_data < threshold)
        left_idx = left_idx.reshape((len(left_idx),))
        left_labels = labels[left_idx]
        left_data = data[left_idx]
        right_idx = np.delete(np.arange(len(feature_data)), left_idx)
        right_labels = labels[right_idx]
        right_data = data[right_idx]
        return left_data, left_labels, right_data, right_labels
        
    def train(self, data, labels):
        """Grows a decision tree by constructing nodes."""
        self.root_node = self.createNodes(data, labels, 0)
    
    def createNodes(self, data, labels, depth):
        """Helper function for train"""
        count1 = sum(labels)
        count0 = len(labels)-sum(labels)
        entropy = self.entropy(count1, count0)
        split_rule = self.segmenter(data, labels)
        if depth == self.max_depth or entropy < self.entropy_threshold or split_rule == None or len(labels) == 1:
            leaf = Node(depth)
            leaf.is_leaf = True
            leaf.set_counts(count1, count0)
            return leaf
        else:
            node = Node(depth)
            left_data, left_labels, right_data, right_labels = self.splitData(split_rule, data, labels)
            node.set_split_rule(split_rule)
            left_node = self.createNodes(left_data, left_labels, depth+1)
            right_node = self.createNodes(right_data, right_labels, depth+1)
            node.set_left(left_node)
            node.set_right(right_node)
            return node
            
    def predict(self, data):
        """Given a data point, traverse the tree to find the best label to classify the data point as."""
        node = self.root_node
        while not node.is_leaf:
            split_rule = node.get_split_rule()
            feature, threshold = split_rule
            if data[feature] < threshold:
                node = node.get_left()
            else:
                node = node.get_right()
        count1, count0 = node.get_counts()
        if count1 > count0:
            return 1
        else:
            return 0

In [96]:
"""RANDOM FOREST"""
class RandomForest:
    def __init__(self, num_trees, max_height, entropy_threshold, training_drop):
        self.num_trees = num_trees
        self.max_height = max_height
        self.entropy_threshold = entropy_threshold
        self.training_drop = training_drop
        self.trees = []
    def train(self, data, labels):
        for tree in range(self.num_trees):
            dt = DecisionTree(self.max_height, self.entropy_threshold)
            sample_idx = np.random.choice(data.shape[0], data.shape[0]-self.training_drop, replace=True)
            data_sample = data[sample_idx]
            labels_sample = labels[sample_idx]
            dt.train(data_sample, labels_sample)
            self.trees.append(dt)
    def predict(self, data):
        """given a data point, outputs ensemble predictions"""
        all_predictions = np.zeros(self.num_trees)
        for idx, dt in enumerate(self.trees):
            prediction = dt.predict(data)
            all_predictions[idx] = prediction
        return stats.mode(all_predictions, axis=0)[0].astype("int")
    def Adaboost_train(self, data, labels):
        """trains a series of trees"""
        curr_data = data
        curr_labels = labels
        for tree in range(self.num_trees):
            print(len(curr_data))
            dt = DecisionTree(self.max_height, self.entropy_threshold)
            dt.train(curr_data, curr_labels)
            predictions = np.array([dt.predict(curr_data[i]) for i in range(curr_data.shape[0])])
            print("accuracy: {0}".format(metrics.accuracy_score(curr_labels, predictions)))
            error_idx = np.argwhere(predictions != curr_labels)
            error_idx = error_idx.reshape((len(error_idx)))
            curr_data = curr_data[error_idx]
            curr_labels = curr_labels[error_idx]
            self.trees.append(dt)

In [113]:
rf = RandomForest(30, 20, 0.01, 2000)

In [114]:
rf.train(X_train_cut, labels_train_cut)

In [115]:
pred_labels_train = np.array([rf.predict(X_train_cut[i]) for i in range(X_train_cut.shape[0])])
pred_labels_val = np.array([rf.predict(X_val[i]) for i in range(X_val.shape[0])])

In [117]:
import sklearn.metrics as metrics
print("Train accuracy: {0}".format(metrics.accuracy_score(labels_train_cut, pred_labels_train)))
print("Val accuracy: {0}".format(metrics.accuracy_score(labels_val, pred_labels_val)))

Train accuracy: 0.9369606903163951
Val accuracy: 0.92


20, 15, 0.01, 1000 drop --> 0.897
20, 15, 0.01, 1500 drop --> 0.9216, 0.908
20, 15, 0.01, 2000 drop --> 0.929, 0.919
20, 15, 0.01, 2500 drop --> 0.919, 0.909
20, 20, 0.01, 2000 --> 0.936， 0.914
30, 20, 0.01, 2000 --> 0.9424, 0.92
100, 20, 0.01, 2000 --> 0.9369, 0.92

In [118]:
"""For Final Submission"""
rf_final = RandomForest(30, 20, 0.01, 2000)
rf_final.train(X_train, labels_train)
pred_labels_train = np.array([rf_final.predict(X_train[i]) for i in range(X_train.shape[0])])
pred_labels_test = np.array([rf_final.predict(X_test[i]) for i in range(X_test.shape[0])])
print("Train accuracy: {0}".format(metrics.accuracy_score(labels_train, pred_labels_train)))

Train accuracy: 0.94276875483372


In [125]:
"""CONTEST SUBMISSION"""
import csv
f = open("ashton_teng_spam_predictions2.csv", 'wt')
try:
    writer = csv.writer(f)
    writer.writerow(('Id', 'Category'))
    for i in range(len(pred_labels_test)):
        writer.writerow( (i+1, pred_labels_test[i][0]) )
finally:
    f.close()